In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    data_path: Path
    tokenizer_path: Path
    metric_filename:Path

In [3]:
from bpReviewClassifier.constants import *
from bpReviewClassifier.utils.common import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_model_eval_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([config.root_dir])
        model_eval_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            data_path=Path(config.data_path),
            tokenizer_path=config.tokenizer_path,
            metric_filename=config.metric_filename,
        )
        return model_eval_config

In [5]:
from transformers import  AutoModelForSequenceClassification,AutoTokenizer
from datasets import load_dataset,load_from_disk
from evaluate import load
import torch
import pandas as pd
from tqdm import tqdm
from datasets import Dataset,DatasetDict
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader

c:\Users\AA\anaconda3\envs\BeautyPro\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-01-28 16:20:41,155 : INFO : config : PyTorch version 2.5.1 available.]


In [6]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config
    def tokenize_fun(self,batch):
            device='cuda' if torch.cuda.is_available() else 'cpu'
            tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
    
            tokenized_inputs = tokenizer(batch['text'], truncation=True, padding='max_length',return_tensors='pt')
            return tokenized_inputs
    def collate_fn(self,batch):
                return {
                'input_ids': torch.tensor([item['input_ids'] for item in batch], dtype=torch.long),
                'attention_mask': torch.tensor([item['attention_mask'] for item in batch], dtype=torch.long),
            }
    def eval_acc(self,outputs,labels):
           
            dataloader = DataLoader(outputs, batch_size=32, collate_fn=self.collate_fn)
            model = AutoModelForSequenceClassification.from_pretrained(self.config.model_path)
            all_preds = []
            total_batches = len(dataloader)  # Total number of batches

            for batch in tqdm(dataloader, total=total_batches, desc="Evaluating", unit="batch"):
                # Extract tensors from the batch
                input_ids = batch['input_ids']
            
                attention_mask = batch['attention_mask']

                with torch.no_grad():
                    preds = model(input_ids=input_ids, attention_mask=attention_mask).logits.argmax(dim=-1)
                
                all_preds.extend(preds.tolist())
               # Compute accuracy
            eval_acc = load('accuracy')
            score = eval_acc.compute(predictions=all_preds, references=labels)
            return score
    def loading_dataset(self):
        df=pd.read_json(self.config.data_path,lines=True)
        df.drop(['title','images','asin','parent_asin','user_id','timestamp', 'helpful_vote', 'verified_purchase'],axis=1,inplace=True)
        #one hot encoding our output column that is rating
        #df['rating']=df['rating']-1
        #splitting dataset into train,test,validation
        train_df,temp_df=train_test_split(df,test_size=0.3,random_state=42)
        test_df,val_df=train_test_split(temp_df,test_size=0.5,random_state=42)
        train_df.reset_index(drop=True, inplace=True)
        val_df.reset_index(drop=True, inplace=True)
        test_df.reset_index(drop=True, inplace=True)

        train_dataset = Dataset.from_pandas(train_df)
        val_dataset = Dataset.from_pandas(val_df)
        test_dataset = Dataset.from_pandas(test_df)
        #preparring datasetDict
        dataset_dict = DatasetDict({
            'train': train_dataset,
            'validation': val_dataset,
            'test': test_dataset
            })
        
        dataset_dict=dataset_dict.rename_column('rating','label')
        labels=dataset_dict['test']['label']
        output=dataset_dict['test'].map(self.tokenize_fun,batched=True)
        

         
        result=self.eval_acc(output,labels)
        accuracy_dict=dict(accuracy=result)
        df=pd.DataFrame(accuracy_dict)
        df.to_csv(self.config.metric_filename,index=False)        
        
           

In [7]:
try:
    config=ConfigurationManager()
    mode_eval_config=config.get_model_eval_config()
    model_evaluation=ModelEvaluation(config=mode_eval_config)
    model_evaluation.loading_dataset()
except Exception as e:
    raise e

[2025-01-28 16:20:43,238 : INFO : common : yaml file config\config.yaml loaded successfully]
[2025-01-28 16:20:43,246 : INFO : common : yaml file params.yaml loaded successfully]
[2025-01-28 16:20:43,246 : INFO : common : artifacts Created successfully.]
[2025-01-28 16:20:43,256 : INFO : common : artifacts/model_evaluation Created successfully.]


Evaluating: 100%|██████████| 3289/3289 [28:22:53<00:00, 31.07s/batch]       
